<a href="https://colab.research.google.com/github/john-jehiel/cards-image-classification/blob/main/Layer_wise_unit_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install thop

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from thop import profile

### Utility Functions

In [3]:
# get list of all layers in a model
def get_layers(model):
    test_layers = []
    for name, layer in model.named_modules():
        if len(list(layer.children())) == 0:  # Only include layers without further submodules
            test_layers.append((layer, f"{name} ({layer.__class__.__name__})"))
    return test_layers

In [4]:
# utility function for easily copying list of test layers
def pretty_print(lst):
    print('[')
    for layer in lst:
        print("   ", layer, ",")
    print(']')

### CNN Architecture

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1   = nn.Conv2d(3, 64, 3)
        self.pool1   = nn.MaxPool2d(2,2)
        self.conv2   = nn.Conv2d(64, 64, 3)
        self.pool2   = nn.MaxPool2d(2,2)
        self.conv3   = nn.Conv2d(64, 64, 3)
        self.pool3   = nn.MaxPool2d(2,2)

        self.f1      = nn.Linear(64 * 26 * 26, 128)
        self.f2      = nn.Linear(128, 128)
        self.f3      = nn.Linear(128, 53)


    def forward(self, x):

        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))

        x = x.view(-1, 26 * 26 * 64)

        x = F.relu(self.f1(x))
        x = F.relu(self.f2(x))
        x = self.f3(x)
        return x


In [6]:
model = CNN()

In [7]:
test_layers = get_layers(model)

In [8]:
pretty_print(test_layers)

[
    (Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1)), 'conv1 (Conv2d)') ,
    (MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), 'pool1 (MaxPool2d)') ,
    (Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)), 'conv2 (Conv2d)') ,
    (MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), 'pool2 (MaxPool2d)') ,
    (Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)), 'conv3 (Conv2d)') ,
    (MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), 'pool3 (MaxPool2d)') ,
    (Linear(in_features=43264, out_features=128, bias=True), 'f1 (Linear)') ,
    (Linear(in_features=128, out_features=128, bias=True), 'f2 (Linear)') ,
    (Linear(in_features=128, out_features=53, bias=True), 'f3 (Linear)') ,
]


### EfficientNet Architecture Without Batchnorm

In [9]:
import torchvision.models as models
from torchvision.models.efficientnet import EfficientNet_B0_Weights

class EfficientNet1(nn.Module):
    def __init__(self):
        super(EfficientNet1, self).__init__()
        # Load EfficientNetB0 backbone
        self.base_model = models.efficientnet_b0(pretrained=True)
        self.base_model.features.requires_grad_(False)  # Freeze base model layers

        self.pooling = nn.AdaptiveAvgPool2d(1)

        self.fc1 = nn.Linear(1280, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 53)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.pooling(x).squeeze(-1).squeeze(-1)  # Global average pooling
        x = self.fc1(x)

        x = self.relu(x)
        x = self.fc2(x)
        return self.softmax(x)

In [10]:
model = EfficientNet1()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 93.3MB/s]


In [11]:
test_layers = get_layers(model)

In [12]:
pretty_print(test_layers)

[
    (Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False), 'base_model.features.0.0 (Conv2d)') ,
    (BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 'base_model.features.0.1 (BatchNorm2d)') ,
    (SiLU(inplace=True), 'base_model.features.0.2 (SiLU)') ,
    (Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False), 'base_model.features.1.0.block.0.0 (Conv2d)') ,
    (BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 'base_model.features.1.0.block.0.1 (BatchNorm2d)') ,
    (SiLU(inplace=True), 'base_model.features.1.0.block.0.2 (SiLU)') ,
    (AdaptiveAvgPool2d(output_size=1), 'base_model.features.1.0.block.1.avgpool (AdaptiveAvgPool2d)') ,
    (Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1)), 'base_model.features.1.0.block.1.fc1 (Conv2d)') ,
    (Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1)), 'base_model.features.1.0.block.1.fc2 (Conv2d)') ,
    (SiLU(inplace=Tr

### EfficientNet Architecture With Batchnorm

In [13]:
import torchvision.models as models
from torchvision.models.efficientnet import EfficientNet_B0_Weights

class EfficientNet2(nn.Module):
    def __init__(self):
        super(EfficientNet2, self).__init__()
        # Load EfficientNetB0 backbone
        self.base_model = models.efficientnet_b0(pretrained=True)
        self.base_model.features.requires_grad_(False)  # Freeze base model layers

        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.batch_norm = nn.BatchNorm1d(128)
        self.fc1 = nn.Linear(1280, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 53)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.base_model.features(x)
        x = self.pooling(x).squeeze(-1).squeeze(-1)  # Global average pooling
        x = self.fc1(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.softmax(x)

In [14]:
model = EfficientNet2()

In [15]:
test_layers = get_layers(model)

In [16]:
pretty_print(test_layers)

[
    (Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False), 'base_model.features.0.0 (Conv2d)') ,
    (BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 'base_model.features.0.1 (BatchNorm2d)') ,
    (SiLU(inplace=True), 'base_model.features.0.2 (SiLU)') ,
    (Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False), 'base_model.features.1.0.block.0.0 (Conv2d)') ,
    (BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 'base_model.features.1.0.block.0.1 (BatchNorm2d)') ,
    (SiLU(inplace=True), 'base_model.features.1.0.block.0.2 (SiLU)') ,
    (AdaptiveAvgPool2d(output_size=1), 'base_model.features.1.0.block.1.avgpool (AdaptiveAvgPool2d)') ,
    (Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1)), 'base_model.features.1.0.block.1.fc1 (Conv2d)') ,
    (Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1)), 'base_model.features.1.0.block.1.fc2 (Conv2d)') ,
    (SiLU(inplace=Tr

### Testing Phase

**Unit Tests for CNN**

In [17]:
!pytest -v testing.py --model=cnn

======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: typeguard-4.4.1, anyio-3.7.1
collected 9 items                                                                                  

testing.py::test_mac_computation[layer0-conv1 (Conv2d)] PASSED                               [ 11%]
testing.py::test_mac_computation[layer1-pool1 (MaxPool2d)] PASSED                            [ 22%]
testing.py::test_mac_computation[layer2-conv2 (Conv2d)] PASSED                               [ 33%]
testing.py::test_mac_computation[layer3-pool2 (MaxPool2d)] PASSED                            [ 44%]
testing.py::test_mac_computation[layer4-conv3 (Conv2d)] PASSED                               [ 55%]
testing.py::test_mac_computation[layer5-pool3 (MaxPool2d)] PASSED                            [ 66%]
testing.py::test_mac_computation[laye

**Unit Tests for EfficientNet (without batchnorm)**

In [18]:
!pytest -v testing.py --model=efficientnet1

======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: typeguard-4.4.1, anyio-3.7.1
collected 235 items                                                                                

testing.py::test_mac_computation[layer0-base_model.features.0.0 (Conv2d)] PASSED             [  0%]
testing.py::test_mac_computation[layer1-base_model.features.0.1 (BatchNorm2d)] PASSED        [  0%]
testing.py::test_mac_computation[layer2-base_model.features.0.2 (SiLU)] PASSED               [  1%]
testing.py::test_mac_computation[layer3-base_model.features.1.0.block.0.0 (Conv2d)] PASSED   [  1%]
testing.py::test_mac_computation[layer4-base_model.features.1.0.block.0.1 (BatchNorm2d)] PASSED [  2%]
testing.py::test_mac_computation[layer5-base_model.features.1.0.block.0.2 (SiLU)] PASSED     [  2%]
testing.py::test_mac_computation[l

**Unit Tests for EfficientNet (with batchnorm)**

In [19]:
!pytest -v testing.py --model=efficientnet2

======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.4, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: typeguard-4.4.1, anyio-3.7.1
collected 236 items                                                                                

testing.py::test_mac_computation[layer0-base_model.features.0.0 (Conv2d)] PASSED             [  0%]
testing.py::test_mac_computation[layer1-base_model.features.0.1 (BatchNorm2d)] PASSED        [  0%]
testing.py::test_mac_computation[layer2-base_model.features.0.2 (SiLU)] PASSED               [  1%]
testing.py::test_mac_computation[layer3-base_model.features.1.0.block.0.0 (Conv2d)] PASSED   [  1%]
testing.py::test_mac_computation[layer4-base_model.features.1.0.block.0.1 (BatchNorm2d)] PASSED [  2%]
testing.py::test_mac_computation[layer5-base_model.features.1.0.block.0.2 (SiLU)] PASSED     [  2%]
testing.py::test_mac_computation[l